# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3233000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-3233000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'J.J82.A98632.T52'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "1H", "1N", "2S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '3D',
  'insta_score': 0.3567,
  'expected_score': -78,
  'adjustment': 18},
 {'call': 'PASS',
  'insta_score': 0.6131,
  'expected_score': -162,
  'adjustment': 31}]

In [4]:
bid.samples

['J.J8x.A98xxx.Txx Q.AQT9xx.Jxx.K9x AKx.Kxx.KTx.Q8xx T98xxxxx.x.Q.AJx 0.24957 1H-1N-2S-P-3H-P-4S-P-P-P (9.2)  1H-1N-2S-3D-3H-P-4S-P-P-P (9.2) ',
 'J.J8x.A98xxx.Txx 98x.AKQ9xxx..K9x Ax.Txx.KQJxx.AJx KQTxxxx..Tx.Q8xx 0.23582 1H-1N-2S-P-4H-P-P-P (10.1)  1H-1N-2S-3D-4H-P-P-P (10.1) ',
 'J.J8x.A98xxx.Txx xx.AKQxxx.QT.J8x Q9x.T9x.KJx.AKQ9 AKT8xxx.x.xx.xxx 0.20850 1H-1N-2S-P-3H-P-4S-P-P-P (8.8)  1H-1N-2S-3D-3H-P-3S-P-4H-P-P-P (8.1) ',
 'J.J8x.A98xxx.Txx Qxx.AQxxxx.Q.J9x xx.KT9x.KJT.AKQx AKT98xx..xxx.8xx 0.20784 1H-1N-2S-P-3H-P-4S-P-P-P (9.7)  1H-1N-2S-3D-P-P-P (9.6) ',
 'J.J8x.A98xxx.Txx 8x.QT9xxx.KQx.Ax KQx.AKx.JTx.QJ98 AT9xxxx.x.x.Kxxx 0.20739 1H-1N-2S-P-3H-P-4S-P-P-P (9.1)  1H-1N-2S-3D-3H-P-4S-P-P-P (9.1) ']

In [5]:
auction = ["PAD_START", "PAD_START", "1H", "1N", "2S","3D","3S","PASS","4S"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': '5D', 'insta_score': 0.1046, 'expected_score': 46, 'adjustment': 5},
 {'call': 'PASS',
  'insta_score': 0.863,
  'expected_score': -244,
  'adjustment': 43}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [6]:
bid.samples

['KQ8xxxx..QT.9xxx J.J8x.A98xxx.Txx AT9x.9xxxxx.x.AQ x.AKQT.KJxx.KJ8x 0.25982 1H-1N-2S-3D-3S-P-4S-P-P-P (10.5)  1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (10.8) ',
 'AQTxxxx.x.T.98xx J.J8x.A98xxx.Txx K8x.AK9xx.xx.Qxx 9x.QTxx.KQJx.AKJ 0.25405 1H-1N-2S-3D-3S-P-4S-P-P-P (9.7)  1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (10.1) ',
 'AQT8xxx.x.x.8xxx J.J8x.A98xxx.Txx K9x.KTxxx.KTx.Qx xx.AQ9x.QJx.AKJ9 0.24568 1H-1N-2S-3D-3S-P-4S-P-P-P (8.6)  1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (9.9) ',
 'AQ8xxxx.T.J.98xx J.J8x.A98xxx.Txx Txx.AK9xxx.Qx.Kx K9.Qxx.KTxx.AQJx 0.22895 1H-1N-2S-3D-3S-P-4S-P-P-P (9.5)  1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (9.5) ',
 'AQTxxxxx.x.x.Jxx J.J8x.A98xxx.Txx 98x.AQ9xx.x.KQxx K.KTxx.KQJTx.A98 0.17219 1H-1N-2S-3D-3S-P-4S-P-P-P (11.0)  1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (9.8) ']

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [7]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

J.J8x.A98xxx.Txx x.xxxx.QJxx.Q8xx AKTxxxx.AKQT..AK Q98x.9x.KTx.J9xx
J.J8x.A98xxx.Txx 9x.KTx.QJxxx.Q8x AKQTxxxx.AQ.K.AK 8x.9xxxx.T.J9xxx
J.J8x.A98xxx.Txx x.xxxx.Kxx.98xxx AKQ9xxxx.AQ.Q.AK T8x.KT9x.JTx.QJx
J.J8x.A98xxx.Txx xx.Txxx.KJTx.Q9x AKQ98xx.AQ9x..AK Txx.Kx.Qxx.J8xxx
J.J8x.A98xxx.Txx xx.Txx.QTxx.Q9xx AKQT98xx.AKQ..A8 xx.9xxx.KJx.KJxx
